# Statistical Analysis 


## Load Data

In [2]:
%%capture

import sys
sys.path.append("..")

import mne
import os
import numpy as np
from scipy import stats 
from mne_bids import BIDSPath, read_raw_bids
from matplotlib import pyplot as plt
from src.utils import load_subject, pipeline
from src.plots import plot_ERPs


# Set BIDS path
bids_path = BIDSPath(root='../data', task='jacobsen', suffix='eeg')

# List of all subject BIDS path
all_sub_paths = bids_path.match()

# Create dictionary to store individual subject ephochs and evoked
sub_epochs = {'regular': [], 'random': []}
sub_evokeds = {'regular': [], 'random': []}


for sub_path in all_sub_paths:
        # Load data
        raw = load_subject(sub_path)

        ## Execute filtering + cleaning data pipeline
        raw, epochs = pipeline(
            raw,
            tmin = -1,
            tmax = 1,
            reject_amp = 100e-6, #±μ100 (if commented out doea not reject components)
            perform_ica = False # ICA doesn't drop any componens
        )

        # Save individual subject epochs and evoked 
        sub_epochs['regular'].append(epochs['regular'])
        sub_epochs['random'].append(epochs['random'])
        sub_evokeds['regular'].append(epochs['regular'].average())
        sub_evokeds['random'].append(epochs['random'].average())

## _t_-Test

In [3]:
diff_waves = []
for i, _ in enumerate(all_sub_paths):
    diff_waves.append(mne.combine_evoked([sub_evokeds['regular'][i], sub_evokeds['random'][i]], weights=[1, -1]))



picks = ["PO3", "PO7", "PO4", "PO8"]

y = np.array([np.mean(e.get_data(picks=picks, tmin=-1, tmax=1),axis=1) for e in diff_waves])

# check shape of result
print('shape: ', y.shape)

# t-Test
t, pval = stats.ttest_1samp(y, 0)
print('Difference t = ', str(round(t[0], 2)), 'p = ', str(round(pval[0], 4)))

shape:  (24, 4)
Difference t =  -3.5 p =  0.0019
